---

<h2> Thí nghiệm </h2>

---

In [144]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Genetic_Algorithm import * 

Nhập số liệu tính toán:

In [215]:
items, max_capacity = get_user_input()
get_info()


Thông tin bài toán:

Vật phẩm 1: Weight = 330, Value = 2870
Vật phẩm 2: Weight = 176, Value = 308
Vật phẩm 3: Weight = 2431, Value = 570
Vật phẩm 4: Weight = 2808, Value = 137
Vật phẩm 5: Weight = 1038, Value = 445
Vật phẩm 6: Weight = 3996, Value = 771
Vật phẩm 7: Weight = 3193, Value = 108
Vật phẩm 8: Weight = 805, Value = 469
Vật phẩm 9: Weight = 627, Value = 3678
Vật phẩm 10: Weight = 2257, Value = 2760
Vật phẩm 11: Weight = 1649, Value = 842
Vật phẩm 12: Weight = 1210, Value = 1237
Vật phẩm 13: Weight = 2642, Value = 1829
Vật phẩm 14: Weight = 508, Value = 967
Vật phẩm 15: Weight = 1192, Value = 313
Vật phẩm 16: Weight = 3446, Value = 1322
Vật phẩm 17: Weight = 3626, Value = 651
Vật phẩm 18: Weight = 1706, Value = 679
Vật phẩm 19: Weight = 3068, Value = 2254
Vật phẩm 20: Weight = 2977, Value = 525
Vật phẩm 21: Weight = 1688, Value = 1571
Vật phẩm 22: Weight = 709, Value = 697
Vật phẩm 23: Weight = 3522, Value = 2262
Vật phẩm 24: Weight = 85, Value = 3065
Vật phẩm 25: Weight = 335

<h2> So sánh kĩ thuật:

<h3> Roulette-wheel Slection và Elitism Selection</h3>

In [220]:
def select_population_roulette(population):
    size_to_select = min(POPULATION_SIZE, len(population))
    fitness_values = [fitness(x) for x in population]
    total_fitness = sum(fitness_values)
    
    # Tránh chia cho 0 nếu tổng fitness = 0
    if total_fitness == 0:
        return random.sample(population, size_to_select)
    
    # Tính xác suất cho mỗi cá thể
    probabilities = [f/total_fitness for f in fitness_values]
    selected_population = []
    
    for _ in range(size_to_select):
        pick = random.uniform(0, 1)
        cumulative_probability = 0
        for i, prob in enumerate(probabilities):
            cumulative_probability += prob
            if pick <= cumulative_probability:
                selected_population.append(population[i])
                break
    return selected_population

# Danh sách các hàm chọn lọc
selection_strategies = {
    "Elitism Selection": select_population,
    "Roulette-wheel Selection": select_population_roulette
}

experiment_results = {}

def run_experiment_with_selection(selection_func):
    global fitness_history, select_population

    # Lưu giá trị hàm select_population ban đầu
    original_select_population = select_population

    # Thay thế hàm select_population cho thử nghiệm
    select_population = selection_func

    # Thực hiện thuật toán di truyền
    best_solution, best_fitness = genetic_algorithm()

    # Lưu kết quả
    result = {
        'best_solution': best_solution,
        'best_fitness': best_fitness,
        'fitness_history': fitness_history.copy()  # Lưu lại lịch sử fitness
    }

    # Reset fitness_history sau mỗi lần thử nghiệm
    fitness_history.clear()

    # Khôi phục hàm select_population ban đầu
    select_population = original_select_population

    return result

# Thực hiện các thử nghiệm cho từng chiến lược chọn lọc
for strategy_name, selection_func in selection_strategies.items():
    print(f"\nThử nghiệm với chiến lược chọn lọc: {strategy_name}")
    
    # Chạy thử nghiệm
    result = run_experiment_with_selection(selection_func)
    
    # Lưu kết quả vào experiment_results
    experiment_results[strategy_name] = result
    
    print(f"Best Solution: {result['best_solution']}")
    print(f"Best Fitness: {result['best_fitness']}")
    print(f"Fitness History: {result['fitness_history']}")



Thử nghiệm với chiến lược chọn lọc: Elitism Selection
Best Solution: [1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0]
Best Fitness: 46538
Fitness History: [39918, 41386, 40448, 39807, 40858, 40858, 41160, 41900, 41900, 39674, 36351, 38279, 38399, 39493, 39486, 41250, 39928, 40970, 40664, 42582, 39875, 40898, 43730, 40911, 43728, 41629, 44712, 40504, 38244, 38858, 37881, 38675, 38326, 39320, 43179, 40045, 42409, 42220, 40400, 40348, 40932, 40932, 44589, 43939, 41621, 41822, 41981, 41471, 41107, 45381, 38939, 41679, 41575, 42867, 42941, 42625, 44842, 46126, 41745, 41852, 43584, 43584, 41320, 43584, 43463, 40680, 43142, 43142, 43378, 42178, 42178, 42178, 42178, 38708, 40264, 41959, 40973, 40028, 42594, 40847, 39610, 39610, 41803, 39779, 39779, 37283, 39709, 40365, 39858, 42109, 42109, 42816, 42816, 42332, 38610, 41507, 43045, 43885, 41722, 41722, 42600, 43442, 44207, 46227, 45939, 43282, 40960, 43594, 45600, 45600

 -Chạy nhiều lần để cho ra các thông số trung bình

In [221]:
import numpy as np

# Số lần chạy thử nghiệm
num_runs = 10

# Dictionary chứa các chiến lược chọn lọc
selection_strategies = {
    "Elitism Selection": select_population,
    "Roulette-wheel Selection": select_population_roulette
}

# Lưu kết quả tổng hợp
aggregated_results = {}

# Thực hiện thử nghiệm cho từng chiến lược chọn lọc
for strategy_name, selection_func in selection_strategies.items():
    fitness_final_list = []
    convergence_gen_list = []
    stability_std_list = []

    print(f"\nChiến lược chọn lọc: {strategy_name}")
    
    for run in range(num_runs):
        # Chạy thử nghiệm với chiến lược chọn lọc hiện tại
        result = run_experiment_with_selection(selection_func)
        history = result['fitness_history']
        final_fitness = history[-1]

        # Tính các giá trị thống kê
        fitness_final_list.append(final_fitness)
        # Tìm thế hệ hội tụ (khi đạt 95% fitness cuối cùng)
        convergence_gen = next((i for i, x in enumerate(history) if x >= 0.95 * final_fitness), len(history))
        convergence_gen_list.append(convergence_gen)
        # Tính độ lệch chuẩn của lịch sử fitness để đánh giá độ ổn định
        stability_std = np.std(history)
        stability_std_list.append(stability_std)

    # Lưu kết quả trung bình và độ lệch chuẩn cho chiến lược hiện tại
    aggregated_results[strategy_name] = {
        "average_fitness": np.mean(fitness_final_list),
        "std_fitness": np.std(fitness_final_list),
        "average_convergence_gen": np.mean(convergence_gen_list),
        "std_convergence_gen": np.std(convergence_gen_list),
        "average_stability_std": np.mean(stability_std_list),
        "std_stability": np.std(stability_std_list)
    }

    # In ra kết quả chi tiết
    print(f"- Fitness trung bình: {aggregated_results[strategy_name]['average_fitness']:.2f} ")
    print(f"- Tốc độ hội tụ trung bình (thế hệ): {aggregated_results[strategy_name]['average_convergence_gen']:.2f} ")
    print(f"- Độ ổn định trung bình: {aggregated_results[strategy_name]['average_stability_std']:.2f} ")



Chiến lược chọn lọc: Elitism Selection
- Fitness trung bình: 40717.60 
- Tốc độ hội tụ trung bình (thế hệ): 2.50 
- Độ ổn định trung bình: 2223.50 

Chiến lược chọn lọc: Roulette-wheel Selection
- Fitness trung bình: 42598.00 
- Tốc độ hội tụ trung bình (thế hệ): 6.10 
- Độ ổn định trung bình: 2087.74 


<h3>One-point crossover vs Uniform Crossover</h3>

In [226]:
def crossover_uniform(parent1, parent2, probability=0.5):
    child1, child2 = parent1[:], parent2[:]
    for i in range(len(parent1)):
        coin_flip = random.choice([0, 1])
        if coin_flip == 0:
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        else:
            child1[i] = parent2[i]
            child2[i] = parent1[i]
    return child1, child2

# Danh sách các hàm lai ghép
crossover_strategies = {
    "One-point Crossover": crossover,  # Lai ghép một điểm
    "Uniform Crossover": crossover_uniform  # Lai ghép đều
}

experiment_results = {}

# Hàm thực hiện thử nghiệm với từng chiến lược lai ghép
def run_experiment_with_crossover(crossover_func):
    global fitness_history, crossover

    # Lưu giá trị hàm crossover ban đầu
    original_crossover = crossover

    # Thay thế hàm crossover cho thử nghiệm
    crossover = crossover_func

    # Thực hiện thuật toán di truyền
    best_solution, best_fitness = genetic_algorithm()

    # Lưu kết quả
    result = {
        'best_solution': best_solution,
        'best_fitness': best_fitness,
        'fitness_history': fitness_history.copy()  # Lưu lại lịch sử fitness
    }

    # Reset fitness_history sau mỗi lần thử nghiệm
    fitness_history.clear()

    # Khôi phục hàm crossover ban đầu
    crossover = original_crossover

    return result

# Thực hiện các thử nghiệm cho từng chiến lược lai ghép
for strategy_name, crossover_func in crossover_strategies.items():
    print(f"\nThử nghiệm với chiến lược lai ghép: {strategy_name}")
    
    # Chạy thử nghiệm
    result = run_experiment_with_crossover(crossover_func)
    
    # Lưu kết quả vào experiment_results
    experiment_results[strategy_name] = result
    
    print(f"Best Solution: {result['best_solution']}")
    print(f"Best Fitness: {result['best_fitness']}")
    print(f"Fitness History: {result['fitness_history']}")


Thử nghiệm với chiến lược lai ghép: One-point Crossover
Best Solution: [1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0]
Best Fitness: 48116
Fitness History: [39207, 39207, 38240, 38311, 38688, 42986, 43123, 42718, 42340, 40513, 43501, 44452, 42300, 42300, 40761, 41641, 41504, 41681, 42832, 42873, 43140, 42130, 43521, 43930, 43641, 41636, 42362, 40132, 45621, 45621, 42099, 44428, 41509, 42907, 41617, 41394, 41629, 42198, 41320, 41730, 41742, 40250, 39450, 38606, 40664, 44659, 43847, 42462, 41928, 45028, 44981, 44566, 40880, 43394, 43351, 38187, 43226, 39918, 39933, 42668, 40475, 41727, 40472, 39891, 39891, 41464, 44368, 42081, 42081, 43057, 43057, 41349, 41193, 40995, 41894, 42573, 40034, 42186, 41199, 43608, 42584, 43826, 43870, 43870, 40965, 41994, 42590, 40744, 41097, 41167, 42887, 42527, 48014, 47877, 40432, 43010, 43069, 45066, 43346, 40825, 39208, 39456, 44216, 40802, 40588, 39826, 44607, 43829, 41290, 422

 -Chạy nhiều lần để cho ra các thông số trung bình

In [227]:
import numpy as np

# Số lần chạy thử nghiệm
num_runs = 10

# Dictionary chứa các chiến lược lai ghép
crossover_strategies = {
    "One-point Crossover": crossover,
    "Uniform Crossover": crossover_uniform
}

# Lưu kết quả tổng hợp
aggregated_results = {}

# Thực hiện thử nghiệm cho từng chiến lược lai ghép
for strategy_name, crossover_func in crossover_strategies.items():
    fitness_final_list = []
    convergence_gen_list = []
    stability_std_list = []
    
    print(f"\nChiến lược lai ghép: {strategy_name}")
        
    for run in range(num_runs):
        # Chạy thử nghiệm với chiến lược lai ghép hiện tại
        result = run_experiment_with_crossover(crossover_func)
        history = result['fitness_history']
        final_fitness = history[-1]
        
        # Tính các giá trị thống kê
        fitness_final_list.append(final_fitness)
        convergence_gen = next((i for i, x in enumerate(history) if x >= 0.95 * final_fitness), len(history))
        convergence_gen_list.append(convergence_gen)
        stability_std = np.std(history)
        stability_std_list.append(stability_std)
    
    # Lưu kết quả trung bình cho chiến lược hiện tại
    aggregated_results[strategy_name] = {
        "average_fitness": np.mean(fitness_final_list),
        "average_convergence_gen": np.mean(convergence_gen_list),
        "average_stability_std": np.mean(stability_std_list),
    }
    
    # In ra kết quả
    print(f"- Fitness trung bình: {aggregated_results[strategy_name]['average_fitness']:.2f}")
    print(f"- Tốc độ hội tụ trung bình: {aggregated_results[strategy_name]['average_convergence_gen']:.2f}")
    print(f"- Độ ổn định trung bình: {aggregated_results[strategy_name]['average_stability_std']:.2f}")


Chiến lược lai ghép: One-point Crossover
- Fitness trung bình: 42097.00
- Tốc độ hội tụ trung bình: 4.80
- Độ ổn định trung bình: 2151.89

Chiến lược lai ghép: Uniform Crossover
- Fitness trung bình: 41647.70
- Tốc độ hội tụ trung bình: 1.10
- Độ ổn định trung bình: 2189.19


<h3>Bit-flip Mutation và Scramble Mutation </h3>

In [232]:
def Scramble_mutate(individual):
   mutated_individual = individual[:]
   for i in range(len(individual)):
       if random.random() < MUTATION_RATE:
           j = random.randint(0, len(individual) - 1)
           mutated_individual[i], mutated_individual[j] = mutated_individual[j], mutated_individual[i]
   return mutated_individual
# Danh sách các hàm đột biến
mutation_strategies = {
    "Bit-flip Mutation": mutate,  # Đột biến Bit-flip
    "Scramble Mutation": Scramble_mutate  # Đột biến Scramble
}

experiment_results = {}

# Hàm thực hiện thử nghiệm với từng chiến lược đột biến
def run_experiment_with_mutation(mutation_func):
    global fitness_history, mutate

    # Lưu giá trị hàm mutate ban đầu
    original_mutate = mutate

    # Thay thế hàm mutate cho thử nghiệm
    mutate = mutation_func

    # Thực hiện thuật toán di truyền
    best_solution, best_fitness = genetic_algorithm()

    # Lưu kết quả
    result = {
        'best_solution': best_solution,
        'best_fitness': best_fitness,
        'fitness_history': fitness_history.copy()  # Lưu lại lịch sử fitness
    }

    # Reset fitness_history sau mỗi lần thử nghiệm
    fitness_history.clear()

    # Khôi phục hàm mutate ban đầu
    mutate = original_mutate

    return result

# Thực hiện các thử nghiệm cho từng chiến lược đột biến
for strategy_name, mutation_func in mutation_strategies.items():
    print(f"\nThử nghiệm với chiến lược đột biến: {strategy_name}")
    
    # Chạy thử nghiệm
    result = run_experiment_with_mutation(mutation_func)
    
    # Lưu kết quả vào experiment_results
    experiment_results[strategy_name] = result
    
    print(f"Best Solution: {result['best_solution']}")
    print(f"Best Fitness: {result['best_fitness']}")
    print(f"Fitness History: {result['fitness_history']}")


Thử nghiệm với chiến lược đột biến: Bit-flip Mutation
Best Solution: [1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0]
Best Fitness: 49456
Fitness History: [39855, 40180, 39979, 42849, 41302, 39731, 40281, 41622, 40730, 40721, 40721, 42292, 39367, 39005, 41334, 43083, 39651, 42892, 44047, 44012, 42772, 43567, 43567, 43992, 41864, 42608, 41346, 43586, 41969, 41556, 42539, 42516, 41855, 43349, 41934, 41582, 41793, 40169, 41007, 41007, 42984, 43337, 42767, 42459, 42868, 40239, 41562, 40391, 41097, 40652, 39254, 40513, 40513, 43264, 39475, 39040, 41052, 40955, 43410, 42206, 41247, 43882, 43914, 46294, 45271, 45271, 40660, 41932, 44450, 45555, 42930, 41507, 39576, 36512, 41546, 42329, 38126, 43867, 43867, 39146, 41785, 46214, 42841, 42888, 42817, 43756, 42141, 42610, 44414, 46009, 43943, 46536, 40446, 42049, 42813, 43075, 42813, 43006, 44965, 41906, 40407, 44342, 43132, 42699, 46717, 46717, 46878, 44415, 44415, 46878

- Chạy nhiều lần để cho ra các thông số trung bình

In [233]:
import numpy as np

# Số lần chạy thử nghiệm
num_runs = 10

# Dictionary chứa các chiến lược đột biến
mutation_strategies = {
    "Bit-flip Mutation": mutate,
    "Scramble Mutation": Scramble_mutate
}

# Lưu kết quả tổng hợp
aggregated_results = {}

# Thực hiện thử nghiệm cho từng chiến lược đột biến
for strategy_name, mutation_func in mutation_strategies.items():
    fitness_final_list = []
    convergence_gen_list = []
    stability_std_list = []

    print(f"\nChiến lược đột biến: {strategy_name}")
    
    for run in range(num_runs):
        # Chạy thử nghiệm với chiến lược đột biến hiện tại
        result = run_experiment_with_mutation(mutation_func)
        history = result['fitness_history']
        final_fitness = history[-1]

        # Tính các giá trị thống kê
        fitness_final_list.append(final_fitness)
        convergence_gen = next((i for i, x in enumerate(history) if x >= 0.95 * final_fitness), len(history))
        convergence_gen_list.append(convergence_gen)
        stability_std = np.std(history)
        stability_std_list.append(stability_std)

    # Lưu kết quả trung bình cho chiến lược hiện tại
    aggregated_results[strategy_name] = {
        "average_fitness": np.mean(fitness_final_list),
        "average_convergence_gen": np.mean(convergence_gen_list),
        "average_stability_std": np.mean(stability_std_list),
    }

    # In ra kết quả
    print(f"- Fitness trung bình: {aggregated_results[strategy_name]['average_fitness']:.2f}")
    print(f"- Tốc độ hội tụ trung bình: {aggregated_results[strategy_name]['average_convergence_gen']:.2f}")
    print(f"- Độ ổn định trung bình: {aggregated_results[strategy_name]['average_stability_std']:.2f}")


Chiến lược đột biến: Bit-flip Mutation
- Fitness trung bình: 41659.50
- Tốc độ hội tụ trung bình: 1.40
- Độ ổn định trung bình: 2210.50

Chiến lược đột biến: Scramble Mutation
- Fitness trung bình: 42216.00
- Tốc độ hội tụ trung bình: 2.50
- Độ ổn định trung bình: 2156.77


---

<h2> Tổng hợp </h3>  

---

Các Tình huống :
- Chạy nhiều lần mỗi case từ 10 lần trở lên .
- Chạy hết 6 testcase trong Testcase.txt
- Tính trung bình cộng toàn bộ thông số bao gồm: fitness cuối cùng, tốc độ hội tụ, độ ổn định sau số lần chạy tuỳ ý.

    - TestCase 1:

| Kỹ thuật ảnh hưởng     | Fitness cuối cùng | Tốc độ hội tụ (thế hệ) | Độ ổn định (std) |
|------------------------|-------------------|-----------------------|------------------|
| **Elitism Selection**  | 49.6            | 1                    | 3.18             |
| **Roulette-wheel Selection**      | 49.9             | 1                     | 3.20              |
|------------------------|-------------------|-----------------------|------------------|
| **One-point Crossover** | 48.70             | 3                     | 3.06              |
| **Uniform Crossover**| 51.2            | 1                     | 3.17              |
|------------------------|-------------------|-----------------------|------------------|
| **Bit-flip Mutation**   | 49.10               | 1                     | 3.31             |
| **Scramble Mutation**       | 51.90             | 2                     | 3.17             |

    - TestCase 2:

| Kỹ thuật ảnh hưởng     | Fitness cuối cùng | Tốc độ hội tụ (thế hệ) | Độ ổn định (std) |
|------------------------|-------------------|-----------------------|------------------|
| **Elitism Selection**  | 58.82            | 1                    | 4.13           |
| **Roulette-wheel Selection**      | 60.06             | 1                     | 4.55              |
|------------------------|-------------------|-----------------------|------------------|
| **One-point Crossover** | 58.70             | 1                     | 4.10              |
| **Uniform Crossover**| 59.10           | 3                     | 4.14              |
|------------------------|-------------------|-----------------------|------------------|
| **Bit-flip Mutation**   | 58.00               | 1                     | 4.08            |
| **Scramble Mutation**       | 60.90             | 1                     | 3.99             |

    - TestCase 3:

| Kỹ thuật ảnh hưởng     | Fitness cuối cùng | Tốc độ hội tụ (thế hệ) | Độ ổn định (std) |
|------------------------|-------------------|-----------------------|------------------|
| **Elitism Selection**  | 2393.4            | 1                    | 82.34             |
| **Roulette-wheel Selection**      | 2862.3              | 1                     | 81.43              |
|------------------------|-------------------|-----------------------|------------------|
| **One-point Crossover** | 2847.6            | 1                     | 84.23              |
| **Uniform Crossover**| 2898.3           | 4                     | 81.27              |
|------------------------|-------------------|-----------------------|------------------|
| **Bit-flip Mutation**   | 2825.0 | 2                     | 86.03            |
| **Scramble Mutation**       | 2883.8            | 2                     | 81.17           |

    - TestCase 4:
    

| Kỹ thuật ảnh hưởng     | Fitness cuối cùng | Tốc độ hội tụ (thế hệ) | Độ ổn định (std) |
|------------------------|-------------------|-----------------------|------------------|
| **Elitism Selection**  | 6904.3            | 2                    | 179.69           |
| **Roulette-wheel Selection**      | 6868.90             | 2                     | 173.04             |
|------------------------|-------------------|-----------------------|------------------|
| **One-point Crossover** | 6859.2             | 3                     | 174.03             |
| **Uniform Crossover**| 6819.1           | 1                     | 177.67              |
|------------------------|-------------------|-----------------------|------------------|
| **Bit-flip Mutation**   | 6821.2              | 3                     | 178.73             |
| **Scramble Mutation**       | 7028.1             | 2                     | 180.31           |

    - TestCase 5:

| Kỹ thuật ảnh hưởng     | Fitness cuối cùng | Tốc độ hội tụ (thế hệ) | Độ ổn định (std) |
|------------------------|-------------------|-----------------------|------------------|
| **Elitism Selection**  | 64421.1           | 1                    | 2832.44             |
| **Roulette-wheel Selection**      | 64778.7             | 2                     | 2704.99              |
|------------------------|-------------------|-----------------------|------------------|
| **One-point Crossover** | 63908.6             | 3                     | 2754.93              |
| **Uniform Crossover**| 66024.2            | 2                     | 2946.92             |
|------------------------|-------------------|-----------------------|------------------|
| **Bit-flip Mutation**   | 62581.8              | 1                     | 2708.96            |
| **Scramble Mutation**       | 64804.5            | 1                     | 2770.95             |

    - TestCase 6:

| Kỹ thuật ảnh hưởng     | Fitness cuối cùng | Tốc độ hội tụ (thế hệ) | Độ ổn định (std) |
|------------------------|-------------------|-----------------------|------------------|
| **Elitism Selection**  | 45437.6            |     9              |2623.5             |
| **Roulette-wheel Selection**      | 48798             | 8                   | 2437.74             |
|------------------------|-------------------|-----------------------|------------------|
| **One-point Crossover** | 42287.5             | 15                     | 2231.89              |
| **Uniform Crossover**| 41767.7          | 10                     | 2449.19             |
|------------------------|-------------------|-----------------------|------------------|
| **Bit-flip Mutation**   | 42336.2            | 10                    | 2750.5             |
| **Scramble Mutation**       | 46321.9             | 12                    |  2836.77            |

<h3> 2. Đánh giá </h3>

- Elitism Selection:

Fitness cuối cùng đạt mức 55.9, cho thấy rằng kỹ thuật này giúp duy trì các cá thể tốt nhất qua các thế hệ.
Tốc độ hội tụ khá nhanh (2 thế hệ), điều này chứng tỏ thuật toán có khả năng tìm kiếm giải pháp tối ưu nhanh chóng.
Độ ổn định là 0.53, cho thấy độ biến thiên trong các giải pháp không quá cao, giúp ổn định trong quá trình tiến hóa.

- One-point Crossover:

Fitness cuối cùng tương đương với Elitism Selection (55.9), cho thấy khả năng sinh ra các cá thể tốt không hề thua kém.
Tốc độ hội tụ là 2 thế hệ, giống như Elitism Selection, cho thấy khả năng tìm kiếm khá nhanh chóng.
Độ ổn định là 0.5, có vẻ như mức độ ổn định tương đương với Elitism Selection, với một mức độ biến thiên không quá lớn.

- Bit-flip Mutation:

Fitness cuối cùng là 56, cao nhất trong bảng, cho thấy kỹ thuật này giúp cải thiện chất lượng các giải pháp.
Tốc độ hội tụ là 3 thế hệ, chậm hơn một chút so với các kỹ thuật chọn lọc và lai ghép, điều này có thể chỉ ra rằng việc thay đổi ngẫu nhiên bit đôi khi không dẫn đến sự tiến hóa nhanh chóng.
Độ ổn định là 0.65, cao nhất, cho thấy kỹ thuật này tạo ra độ biến thiên cao hơn trong quá trình tiến hóa, có thể là do tác động mạnh mẽ của sự thay đổi bit.